In [83]:
import pandas as pd
import numpy as np
import sys
import math
import folium
from folium.plugins import HeatMap
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import scipy.stats as stats
from xml.dom.minidom import parse, parseString
import matplotlib.pyplot as plt
from math import cos, asin, sqrt, pi

In [84]:
# give a sample photo
photo = "IMG_1435.JPG"

In [115]:
def get_exif(filename):
    return Image.open(filename)._getexif()

# test
exif = get_exif(photo)

In [86]:
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

geotags = get_geotagging(exif)
geotags

{'GPSLatitudeRef': 'N',
 'GPSLatitude': (49.0, 15.0, 32.96),
 'GPSLongitudeRef': 'W',
 'GPSLongitude': (122.0, 53.0, 16.09),
 'GPSAltitudeRef': b'\x00',
 'GPSAltitude': 110.72949032433905,
 'GPSSpeedRef': 'K',
 'GPSSpeed': 0.0,
 'GPSImgDirectionRef': 'T',
 'GPSImgDirection': 268.4230649673609,
 'GPSDestBearingRef': 'T',
 'GPSDestBearing': 268.4230649673609,
 'GPSHPositioningError': 65.0}

In [101]:
def get_decimal_from_dms(dms, ref):
    degrees = dms[0]
    minutes = dms[1] / 60.0
    seconds = dms[2] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)

lat, lon = get_coordinates(geotags)
print(lat)
print(lon)

49.25916
-122.8878


In [88]:
def cal_distance(hotel, lat, lon):
    p = pi / 180
    lat1 = lat 
    lon1 = lon 
    lat2 = hotel['latitude']
    lon2 = hotel['longitude']

    a = 0.5-np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    b = 12742*np.arcsin(np.sqrt(a))*1000
   
    return b

In [89]:
#Read Data and Clean Unnecassary information
hotel = pd.read_csv("listings.csv")
hotel = hotel.drop(['host_id','neighbourhood_group','last_review','reviews_per_month','calculated_host_listings_count' ], axis = 1)
hotel

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365
0,10080,D1 - Million Dollar View 2 BR,Rami,Downtown,49.288720,-123.120460,Entire home/apt,150,90,16,339
1,13358,Monthly (or Longer ) Designer One Bedroom Down...,Lynn,West End,49.282010,-123.126690,Entire home/apt,85,30,430,61
2,13490,Vancouver's best kept secret,Iris,Kensington-Cedar Cottage,49.256220,-123.066070,Entire home/apt,140,30,89,195
3,14267,EcoLoft Vancouver,Peter & Alison,Kensington-Cedar Cottage,49.249220,-123.081390,Entire home/apt,150,3,33,2
4,16254,Close to PNE/Hastings Park Garden level suite,Jason,Hastings-Sunrise,49.277210,-123.040860,Entire home/apt,195,3,5,310
...,...,...,...,...,...,...,...,...,...,...,...
4325,50833831,2br-1150 sqft \n4 flrs Furnished Row house Gas...,Patricia,Downtown,49.284466,-123.102002,Entire home/apt,103,31,0,258
4326,50834014,Piplup5,Rachel,Marpole,49.218155,-123.130833,Private room,35,1,0,14
4327,50842488,1168 Pendrell S - 2 Bedrooms - Downtown Vancouver,Plan A,West End,49.281144,-123.132688,Entire home/apt,147,30,0,62
4328,50854207,1170 Barclay Street - Brand new Studio,Plan A,West End,49.285058,-123.128492,Entire home/apt,75,30,0,97


In [90]:
distance = cal_distance(hotel, lat, lon)
hotel['distance'] = distance
hotel

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
0,10080,D1 - Million Dollar View 2 BR,Rami,Downtown,49.288720,-123.120460,Entire home/apt,150,90,16,339,17196.154769
1,13358,Monthly (or Longer ) Designer One Bedroom Down...,Lynn,West End,49.282010,-123.126690,Entire home/apt,85,30,430,61,17517.494318
2,13490,Vancouver's best kept secret,Iris,Kensington-Cedar Cottage,49.256220,-123.066070,Entire home/apt,140,30,89,195,12941.584564
3,14267,EcoLoft Vancouver,Peter & Alison,Kensington-Cedar Cottage,49.249220,-123.081390,Entire home/apt,150,3,33,2,14093.664170
4,16254,Close to PNE/Hastings Park Garden level suite,Jason,Hastings-Sunrise,49.277210,-123.040860,Entire home/apt,195,3,5,310,11285.454910
...,...,...,...,...,...,...,...,...,...,...,...,...
4325,50833831,2br-1150 sqft \n4 flrs Furnished Row house Gas...,Patricia,Downtown,49.284466,-123.102002,Entire home/apt,103,31,0,258,15793.372483
4326,50834014,Piplup5,Rachel,Marpole,49.218155,-123.130833,Private room,35,1,0,14,18223.877018
4327,50842488,1168 Pendrell S - 2 Bedrooms - Downtown Vancouver,Plan A,West End,49.281144,-123.132688,Entire home/apt,147,30,0,62,17934.989318
4328,50854207,1170 Barclay Street - Brand new Studio,Plan A,West End,49.285058,-123.128492,Entire home/apt,75,30,0,97,17698.276275


In [91]:
def choose_byDistance(hotel, dis):
    return hotel[hotel['distance'] < dis]

hotel_dis = choose_byDistance(hotel, 11000)
hotel_dis

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
6,17158,Vancouver 3br 3ba house 20min to DT,Elaine,Renfrew-Collingwood,49.252770,-123.037740,Entire home/apt,185,4,5,1,10905.036759
49,306178,Sunset views - LGBTQ2+ friendly,Michelle,Renfrew-Collingwood,49.232730,-123.027470,Private room,70,2,72,130,10555.936635
59,461370,Lovely large room w/ private bath,Nala,Killarney,49.228340,-123.030470,Private room,84,30,44,365,10909.077475
60,461386,Beautiful room (s) Vancouver Home,Nala,Killarney,49.230340,-123.029710,Private room,84,30,35,365,10788.379098
168,2084273,Gorgeous New House Double Ensuite,Renee,Renfrew-Collingwood,49.249760,-123.035470,Private room,75,31,41,334,10768.294270
...,...,...,...,...,...,...,...,...,...,...,...,...
4229,50424076,Private 1 Bedroom suite in East Vancouver,Min Ming,Hastings-Sunrise,49.268740,-123.031310,Entire home/apt,71,2,0,24,10467.866016
4246,50528364,Modern yet Cozy house in Vancouver w/ 2 bed 2 ...,Sue,Renfrew-Collingwood,49.249510,-123.037090,Entire home/apt,94,30,0,313,10888.051526
4306,50775791,Near Metrotown Joyce skytrain,Yan,Renfrew-Collingwood,49.237674,-123.022643,Private room,45,1,0,332,10075.106197
4311,50780432,Brand New Spacious & Central Home,Kashif,Hastings-Sunrise,49.286057,-123.029172,Entire home/apt,550,2,0,14,10683.712456


In [92]:
def choose_nearest(hotel):
    return hotel[hotel['distance'] == hotel['distance'].min()]

hotel_nearest = choose_nearest(hotel)
hotel_nearest

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
4312,50780970,Be away for summer hot！,Xiuci,Renfrew-Collingwood,49.242392,-123.019537,Entire home/apt,123,2,0,362,9741.870232


In [93]:
def choose_byRegion(hotel, region):
    return hotel[hotel['neighbourhood'] == region]

hotel_region = choose_byRegion(hotel, 'Downtown')
hotel_region

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
0,10080,D1 - Million Dollar View 2 BR,Rami,Downtown,49.288720,-123.120460,Entire home/apt,150,90,16,339,17196.154769
18,26047,The Alpha Downtown Accommodation by COMFYSUITES,Alicia,Downtown,49.280860,-123.123750,Entire home/apt,159,30,35,297,17288.358048
19,32746,Designer Urban Park View Townhouse,Liliana,Downtown,49.279200,-123.105000,Entire home/apt,99,30,80,335,15915.786350
21,35102,Downtown Yaletown Trendy 2 Bd/2Bth,Karina,Downtown,49.277430,-123.118690,Entire home/apt,132,30,76,0,16875.331898
23,42645,The Charlie by COMFYSUITES,Alicia,Downtown,49.280360,-123.123400,Entire home/apt,159,30,30,319,17255.615671
...,...,...,...,...,...,...,...,...,...,...,...,...
4296,50734168,Downtown 2BD/2BA/Den+Amazing View+Skytrain+Par...,Lillian,Downtown,49.279370,-123.108310,Entire home/apt,299,2,0,360,16156.205055
4318,50796676,Cute and Cozy Studio in the Heart of DT/Yaletown,Salim,Downtown,49.276267,-123.131674,Entire home/apt,76,30,0,123,17796.842465
4321,50817305,Beautiful 2BR in Downtown+FreePKG+1mintoHarbour,Ash,Downtown,49.278490,-123.106440,Entire home/apt,257,1,0,335,16008.565541
4322,50823438,Gorgeous View - Private Condo ( WITH AC ),Hoang Yen,Downtown,49.277796,-123.107704,Entire home/apt,220,3,1,256,16089.463669


In [110]:
def choose_byReviews(hotel):
    return hotel[hotel['number_of_reviews'] == hotel['number_of_reviews'].max()]

hotel_review = choose_byReviews(hotel)
hotel_review

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
272,3812348,"Cosy room near scenic park, pvt BR",Catherine,Riley Park,49.2357,-123.10428,Private room,75,2,662,32,15928.761887


In [95]:
def choose_byPrice(hotel, price):
    return hotel[hotel['price'] < price]

hotel_price = choose_byPrice(hotel, 100)
hotel_price

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
1,13358,Monthly (or Longer ) Designer One Bedroom Down...,Lynn,West End,49.282010,-123.126690,Entire home/apt,85,30,430,61,17517.494318
7,17765,"Central location, nice little apartment",James,Mount Pleasant,49.261320,-123.108450,Entire home/apt,94,3,194,0,16014.038737
8,18270,private rm in clean central 2BR apt,Ran,Mount Pleasant,49.265570,-123.096000,Private room,50,30,118,280,15124.913449
9,18589,Commercial Drive B&B,Sylvain & Alexis,Grandview-Woodland,49.275690,-123.070570,Private room,79,1,390,1,13388.185694
11,18795,*Best choice in downtown-5 Stars * -2,Kai,West End,49.282990,-123.130080,Private room,85,30,122,365,17776.628048
...,...,...,...,...,...,...,...,...,...,...,...,...
4316,50795092,Artsy Private Guest Suite with Outdoor Saloon,Jordan,Riley Park,49.248690,-123.105020,Entire home/apt,89,1,1,309,15808.272456
4318,50796676,Cute and Cozy Studio in the Heart of DT/Yaletown,Salim,Downtown,49.276267,-123.131674,Entire home/apt,76,30,0,123,17796.842465
4320,50813864,Private room with shared bathroom near UBC,Liz,Shaughnessy,49.248621,-123.127759,Private room,67,1,0,117,17455.067857
4326,50834014,Piplup5,Rachel,Marpole,49.218155,-123.130833,Private room,35,1,0,14,18223.877018


In [106]:
def choose_lowestPrice(hotel):
    return hotel[hotel['price'] == hotel['price'].min()]

hotel_lowprice = choose_lowestPrice(hotel)
hotel_lowprice

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
2113,32272216,1 bright bdrm with large shared bath,Kitty,Kensington-Cedar Cottage,49.23933,-123.08915,Private room,12,30,3,283,14780.322267


In [108]:
hotel_filter = hotel_price = choose_byPrice(hotel, 100)
hotel_filter = choose_region(hotel_filter, 'Downtown')
hotel_filter = choose_byDistance(hotel_filter, 20000)
hotel_filter

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
19,32746,Designer Urban Park View Townhouse,Liliana,Downtown,49.279200,-123.105000,Entire home/apt,99,30,80,335,15915.786350
46,258636,FABULOUS Luxury Downtown Vancouver!,Ian,Downtown,49.278390,-123.128520,Entire home/apt,99,120,53,97,17596.069495
99,980511,"261 Yaletown Studio at ""Space""",Bruce,Downtown,49.274890,-123.126540,Entire home/apt,70,30,7,46,17410.698929
100,994969,152 Studio (no Bedroom),Bruce,Downtown,49.280060,-123.126730,Entire home/apt,61,30,1,3,17490.576536
105,1021607,Deluxe Junior One Bedroom (2 Beds),Hajime,Downtown,49.277470,-123.120980,Entire home/apt,70,30,100,13,17040.813851
...,...,...,...,...,...,...,...,...,...,...,...,...
4175,50116004,Executive one bed- condo in the heart of YaleT...,Jas,Downtown,49.276855,-123.125221,Entire home/apt,85,30,0,295,17338.574231
4195,50229369,Beautiful 1 bedroom in Yaletown with free parking,Kira,Downtown,49.274680,-123.124710,Entire home/apt,90,30,0,102,17276.282094
4247,50542042,4Months or Longer- City View LUXURY Yaletown C...,Anelia,Downtown,49.276270,-123.114300,Entire home/apt,94,120,0,148,16544.032196
4252,50554509,Brand new 1 Bed + Den close to Water front,Cayden,Downtown,49.277122,-123.128647,Entire home/apt,95,30,0,126,17588.870933


In [112]:
hotel_filter = choose_byReviews(hotel_filter)
hotel_filter

,id,name,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,distance
415,6808688,"Bright room, Amazing View, Heart of Downtown",Laulé,Downtown,49.28945,-123.12525,Private room,61,30,335,141,17552.649138


In [113]:
# define the world map
amen = pd.read_json('amenities-vancouver.json.gz', lines=True)
van_map = folium.Map(location=[lat, lon], zoom_start=12)

incidents = folium.map.FeatureGroup()

folium.Marker(
        location=[lat, lon],
        popup="My postion",
        icon=folium.Icon(icon="home"),
    ).add_to(van_map)
van_map

In [114]:
incidents = folium.map.FeatureGroup()
latitudes = list(hotel_filter.latitude)
longitudes = list(hotel_filter.longitude)
labels = list(hotel_filter.name)
for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(van_map)
van_map.add_child(incidents)

In [99]:
df = pd.DataFrame({'origin_lng' : lon, 'origin_lat' : lat,
                   'destination_lng': hotel_filter['longitude'], 'destination_lat': hotel_filter['latitude']})

print(df.head())

     origin_lng  origin_lat  destination_lng  destination_lat
272   -122.8878     49.2357       -123.10428          49.2357


In [100]:
folium.CircleMarker([row['origin_lat'], row['origin_lng']],
                        radius=15,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(van_map)

folium.CircleMarker([row['destination_lat'], row['destination_lng']],
                        radius=15,
                        fill_color="red", # divvy color
                       ).add_to(van_map)

folium.PolyLine([[row['origin_lat'], row['origin_lng']], 
                     [row['destination_lat'], row['destination_lng']]]).add_to(van_map)
van_map